<a href="https://colab.research.google.com/github/Abhishek-world/abhibajpai/blob/master/Advanced_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd

In [4]:
from google.colab import files
uploaded=files.upload()

Saving u.data to u.data


In [0]:
import io
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(io.BytesIO(uploaded['u.data']), sep='\t', names=column_names)

In [6]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
uploaded=files.upload()

movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

Saving Movie_Id_Titles to Movie_Id_Titles


,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [10]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


Now let's take a quick look at the number of unique users and movies.

In [21]:
n_users=df.item_id.nunique()
n_items=df.user_id.nunique()
print("Number of Unique Users",n_users)
print("Number of Unique Items",u_items)

Number of Unique Users 1682
Number of Unique Items 943


In [15]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.2.post1)


In [17]:
!pip install Cython

In [0]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [32]:
train_data.shape

(75000, 5)

In [33]:
test_data.shape

(25000, 5)

In [34]:
train_data.head()


,user_id,item_id,rating,timestamp,title
38453,10,191,5,877888613,Amadeus (1984)
50954,182,50,5,885613018,Star Wars (1977)
73015,798,283,5,875637963,Emma (1996)
58664,406,655,3,880131704,Stand by Me (1986)
26820,802,452,4,875985976,Jaws 2 (1978)


In [0]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))

for line in train_data.itertuples():
  try:
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  
  except:
    pass
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
  try:
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
  except:
    pass
    

In [0]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [0]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [49]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')
print()


In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [47]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1741526177583332
Item-based CF RMSE: 3.3394964590950678
